# 5. Multivariate Linear Model

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

## 5.1 Prepare Data

@Solubility Data

Tetko et al. (2001) and Huuskonen (2000) investigated a set of compounds with corresponding experimental solubility values using complex sets of descriptors. They used linear regression and neural network models to estimate the relationship between chemical structure and solubility. For our analyses, we will use 1267 compounds and a set of more understandable descriptors that fall into one of three groups: 208 binary "fingerprints" that indicate the presence or absence of a particular chemical sub-structure, 16 count descriptors (such as the number of bonds or the number of Bromine atoms) and 4 continuous descriptors (such as molecular weight or surface area).

1267 observation (951+316)

228 mixed variables

www.rdocumentation.org/packages/AppliedPredictiveModeling/versions/1.1-6/topics/solubility

In [3]:
import pandas as pd

data = {}
f_list = ["solTestX", "solTestY", "solTrainX", "solTrainY"]

for i in f_list :
    data[i] = pd.read_csv("./data/"+i+".csv", index_col=0)
    
print("Data Loaded")

Data Loaded


In [4]:
train_data = data["solTrainX"]
train_label = data["solTrainY"]
test_data = data["solTestX"]
test_label = data["solTestY"]

print("train data : ", train_data.shape)
print("train label : ", train_label.shape)
print("test data : ", test_data.shape)
print("test label : ", test_label.shape)

arr = train_data.columns

train data :  (951, 228)
train label :  (951, 1)
test data :  (316, 228)
test label :  (316, 1)


In [5]:
data["solTestX"].head()

,FP001,FP002,FP003,FP004,FP005,FP006,FP007,FP008,FP009,FP010,...,NumCarbon,NumNitrogen,NumOxygen,NumSulfer,NumChlorine,NumHalogen,NumRings,HydrophilicFactor,SurfaceArea1,SurfaceArea2
20,1,0,0,1,0,0,0,0,1,1,...,1.301297,0.000000,1.098612,0.0,0.375,0.375,0.000000,0.416563,8.245324,8.245324
21,1,0,1,1,0,1,0,0,0,0,...,1.719055,0.456826,1.098612,0.0,0.000,0.000,0.000000,0.915006,9.765805,9.765805
23,0,1,0,0,1,0,0,0,0,1,...,2.642633,0.000000,1.098612,0.0,0.000,0.000,0.693147,0.652185,8.591537,8.591537
25,0,0,1,0,0,1,0,0,0,0,...,2.372566,0.584815,0.000000,0.0,0.000,0.000,0.693147,0.734273,6.568437,6.568437
28,1,1,0,1,1,1,0,1,0,0,...,2.642633,0.584815,0.693147,0.0,0.000,0.000,0.693147,0.650346,10.095894,10.095894


In [6]:
x, y = torch.from_numpy(train_data.values).float(), torch.from_numpy(train_label.values).float()
x, y

(tensor([[ 0.0000,  1.0000,  0.0000,  ..., -1.6065,  6.8125,  6.8125],
         [ 0.0000,  1.0000,  0.0000,  ..., -0.4413,  9.7538, 12.0296],
         [ 1.0000,  1.0000,  1.0000,  ..., -0.3849,  8.2453,  8.2453],
         ...,
         [ 1.0000,  0.0000,  1.0000,  ..., -1.4838,  5.4565,  5.4565],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5332,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  1.0000,  ..., -1.3834,  3.8371,  3.8371]]),
 tensor([[-3.9700e+00],
         [-3.9800e+00],
         [-3.9900e+00],
         [-4.0000e+00],
         [-4.0600e+00],
         [-4.0800e+00],
         [-4.0800e+00],
         [-4.1000e+00],
         [-4.1000e+00],
         [-4.1100e+00],
         [-4.1100e+00],
         [-4.1200e+00],
         [-4.1400e+00],
         [-4.1400e+00],
         [-4.1500e+00],
         [-4.1600e+00],
         [-4.1600e+00],
         [-4.1600e+00],
         [-4.1600e+00],
         [-4.1700e+00],
         [-4.1800e+00],
         [-4.1900e+00],
         [-4.1900e+00],
      

## 5.2 Define Model

In [7]:
model = nn.Linear(228, 1, bias=True)
print(model)

Linear(in_features=228, out_features=1, bias=True)


In [8]:
loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-5)

## 5.3 Train Model

In [9]:
for step in range(50000) :
    pre = model(x)
    cost = loss(pre, y)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if step % 5000 == 0 :
        print("Step : ",step, ", Cost : ", cost.item())

Step :  0 , Cost :  9.87882137298584
Step :  5000 , Cost :  1.9778108596801758
Step :  10000 , Cost :  1.6957393884658813
Step :  15000 , Cost :  1.5376285314559937
Step :  20000 , Cost :  1.4102637767791748
Step :  25000 , Cost :  1.3042882680892944
Step :  30000 , Cost :  1.2150768041610718
Step :  35000 , Cost :  1.1393853425979614
Step :  40000 , Cost :  1.0747268199920654
Step :  45000 , Cost :  1.0191460847854614


In [10]:
model.state_dict()

OrderedDict([('weight',
              tensor([[-2.5280e-02,  1.6148e-02, -1.0149e-02,  8.6722e-03, -3.4660e-03,
                        4.1309e-02,  1.0095e-02,  5.4980e-03, -1.2710e-01,  1.3206e-02,
                        6.4271e-02, -5.3406e-02, -9.1661e-02, -9.2723e-02,  3.2460e-02,
                        3.5549e-02,  3.0283e-02, -1.9389e-02, -1.1520e-02,  6.0760e-02,
                       -1.4497e-02, -6.7233e-02,  4.3476e-02, -1.5004e-02, -4.4013e-02,
                       -2.5398e-03,  5.4671e-02,  4.6868e-02,  4.8212e-03, -5.4781e-02,
                        6.1138e-02,  7.2214e-02,  7.5644e-02,  1.4051e-02, -2.9290e-03,
                       -3.3368e-03, -4.7902e-02, -3.8926e-02, -1.9284e-02,  1.2651e-03,
                        3.4072e-02,  1.2683e-02,  9.1122e-05, -7.4912e-02,  6.5588e-02,
                        8.1160e-02,  2.7687e-02, -8.6889e-02,  3.1578e-02, -1.3279e-03,
                       -1.7379e-02,  3.1191e-03,  6.1387e-02, -5.0030e-02, -1.0943e-02,
        

## 5.4 Test Model

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
x_test = torch.from_numpy(test_data.as_matrix()).float()
y_test = torch.from_numpy(test_label.as_matrix()).float()

C:\Users\slcf\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\slcf\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [13]:
y_predict = model(x_test)

In [14]:
from sklearn import metrics
print("R2 Score = ", metrics.r2_score(y_test.data, y_predict.data))

R2 Score =  0.767331452338332


In [15]:
plt.scatter(y_predict.data.numpy(), y_test.data.numpy())

plt.title('Predict & Real', fontdict={'size':15})
plt.xlabel('Predict')
plt.ylabel('Real')

plt.xlim(-10, 3) 
plt.ylim(-10, 3)

plt.plot(y_test.data.numpy(), y_test.data.numpy(), 'r-') 
plt.show()